In [ ]:
#Dependencies
import ast
import torch
import pandas as pd
from google.colab import files
from transformers import pipeline

In [ ]:
#Set sentiment pipeline
task = "text-classification"
model = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_pipeline = pipeline(task, model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def analyze_sentiments(df, sentiment_pipeline):
    """
    Takes reviews from inspected restaurant and associates them with 'positive'
    or 'negative' sentiment.

    Returns the same dataframe but with an extra column 'sentiment'.
    """
    sentiment_str_per_row = list()
    sentiment_int_per_row = list()

    # Compute the length of the DataFrame
    num_rows = len(df)

    # Output replacements
    output_to_str = {'1 star': 'negative', '2 stars': 'negative', '3 stars': 'neutral', '4 stars': 'positive', '5 stars': 'positive'}
    output_to_int = {'1 star': 1, '2 stars': 2, '3 stars': 3, '4 stars': 4, '5 stars': 5}

    # Traverse dataset's rows
    for index, row in df.iterrows():
      if index % 44 == 0:
        print(f'progress: {round((index+1)/num_rows*100,3)}%')
      stars = [sentiment_pipeline(review[:512])[0]['label'] for review in row['reviews']]
      sentiments_str = [output_to_str[star] for star in stars]
      sentiments_int = [output_to_int[star] for star in stars]
      sentiment_str_per_row.append(sentiments_str)
      sentiment_int_per_row.append(sentiments_int)

    df['sentiment_str'] = sentiment_str_per_row
    df['sentiment_int'] = sentiment_int_per_row

    return df

In [ ]:
# Upload input file
uploaded = files.upload()

Saving labeled_inspections_with_reviews.csv to labeled_inspections_with_reviews.csv


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
filename = 'labeled_inspections_with_reviews.csv'

# Load data
data = pd.read_csv(filename)

# Clean review data
for column in ['reviews', 'ratings']:
    data[column] = data[column].apply(ast.literal_eval)

data.head()

,Unnamed: 0,Organization Name,Active Indicator,Public Facility Name,Program Group Type,Address,City,County Name,Zip Code,State,...,is_open,attributes,categories,hours,prev_date,prev_date_with_nulls,reviews,ratings,n_reviews,avg_rating
0,10735,Chester County Health Department,Yes,"158 LICENSE, LLC",ChesterCountyFood,158 W GAY ST,West Chester,Chester,19380.0,PA,...,0.0,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","Restaurants, American (New)","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",NaN,2018-03-25 00:00:00,[Our first visit here and probably won't be ba...,"[1.0, 3.0, 1.0, 5.0, 1.0, 5.0, 1.0, 2.0, 5.0, ...",11,2.454545
1,56848,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,3 B'S LOUNGE,Food,309 S MAIN ST,ZELIENOPLE,Butler,16063.0,PA,...,1.0,"{'RestaurantsGoodForGroups': 'True', 'Business...","American (Traditional), Bars, Nightlife, Sandw...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'...",NaN,2017-05-21 00:00:00,"[Stopped in for happy hour last night, first t...","[4.0, 5.0]",2,4.500000
2,72731,PENNSYLVANIA DEPARTMENT OF AGRICULTURE,Yes,3 B'S LOUNGE,Food,309 S MAIN ST,ZELIENOPLE,Butler,16063.0,PA,...,1.0,"{'RestaurantsGoodForGroups': 'True', 'Business...","American (Traditional), Bars, Nightlife, Sandw...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'...",2018-05-16 00:00:00,2018-05-16 00:00:00,[Six of us went here for an early Saturday lun...,"[5.0, 1.0, 4.0, 5.0, 1.0, 5.0, 4.0]",7,3.571429
3,289,Chester County Health Department,Yes,30 MAIN,ChesterCountyFood,660 LANCASTER AVE,Berwyn,Chester,19312.0,PA,...,1.0,"{'RestaurantsPriceRange2': '2', 'GoodForMeal':...","American (New), Bars, Venues & Event Spaces, N...","{'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22...",NaN,2017-05-08 00:00:00,[So I read all the reviews and was having seco...,"[2.0, 5.0, 5.0, 3.0, 4.0, 2.0, 1.0, 1.0, 5.0, ...",11,3.363636
4,15080,Chester County Health Department,Yes,30 MAIN,ChesterCountyFood,660 LANCASTER AVE,Berwyn,Chester,19312.0,PA,...,1.0,"{'RestaurantsPriceRange2': '2', 'GoodForMeal':...","American (New), Bars, Venues & Event Spaces, N...","{'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22...",2018-05-03 00:00:00,2018-05-03 00:00:00,"[Ordered through Grub Hub, and the delivery ca...","[1.0, 2.0, 5.0, 2.0, 1.0]",5,2.200000


In [ ]:
#Find review sentiments
data = analyze_sentiments(data, sentiment_pipeline)

progress: 0.046%
progress: 2.079%
progress: 4.111%
progress: 6.143%
progress: 8.176%
progress: 10.208%
progress: 12.24%
progress: 14.273%
progress: 16.305%
progress: 18.337%
progress: 20.37%
progress: 22.402%
progress: 24.434%
progress: 26.467%
progress: 28.499%
progress: 30.531%
progress: 32.564%
progress: 34.596%
progress: 36.628%
progress: 38.661%
progress: 40.693%
progress: 42.725%
progress: 44.758%
progress: 46.79%
progress: 48.822%
progress: 50.855%
progress: 52.887%
progress: 54.919%
progress: 56.952%
progress: 58.984%
progress: 61.016%
progress: 63.048%
progress: 65.081%
progress: 67.113%
progress: 69.145%
progress: 71.178%
progress: 73.21%
progress: 75.242%
progress: 77.275%
progress: 79.307%
progress: 81.339%
progress: 83.372%
progress: 85.404%
progress: 87.436%


In [ ]:
data.head()

In [ ]:
# Export DataFrame to CSV, and download
data.to_csv('labeled_inspections_with_reviews_sentiments.csv', index=False)
files.download('/content/labeled_inspections_with_reviews_sentiments.csv')